In [18]:
%load_ext autoreload
%autoreload 2

import os
import datetime as dt

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

import models
import encoders
import decoders
from training import train, test, checkpoint, main_wandb, main_simple
from dataloaders import load_data, make_loaders, append_SOS
import utils

import sys
sys.path.insert(0, '/scratch/users/udemir15/ELEC491/bassline_transcription')
from utilities import *

from bassline_transcriber.transcription import NN_output_to_MIDI_file

SEED = 27

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
cuda


In [17]:
M = 8

data_params = {'dataset_path': '/scratch/users/udemir15/ELEC491/bassline_transcription/data/datasets/[28, 51]',
               'dataset_name': 'TechHouse_bassline_representations_4020',
               'scale_type': 'min',
               'M': M}

X, titles = load_data(data_params, True)

K = int(X.max()+1) # Number of classes, assumes consecutive [0,max] inclusive
sequence_length = X.shape[1]

print('Number of classes: {}\nSequence Length: {}'.format(K, sequence_length))
print('Number of data points: {}'.format(X.shape[0]))

SOS token added.
Number of classes: 27
Sequence Length: 65
Number of data points: 4020


frequencies = torch.tensor(np.unique(X, return_counts=True)).to(device)
cross_entropy_weights = frequencies[1].max() / frequencies[1]
cross_entropy_weights /= cross_entropy_weights.sum()
print('Cross entropy weights:\n{}\n'.format(cross_entropy_weights))
cross_entropy_weights = torch.FloatTensor(1 / (frequencies[1] / frequencies[1].max())).to(device)
print(cross_entropy_weights)

In [ ]:
train_params = {'batch_size': 32,
               'N_epochs': 150,
               'lr': 5e-4,
               'teacher_forcing_ratio': 0.75}

decoder_hidden_size = 64

encoder_params = {'input_size': K,
                  'embedding_size': 12,
                  'hidden_size': 64,
                  'output_size': decoder_hidden_size}

decoder_params = {'output_size': K,
                  'embedding_size': encoder_params['embedding_size'],
                  'hidden_size': decoder_hidden_size}

params = {'data': data_params,
          'encoder': encoder_params,
          'decoder':decoder_params, 
          'training':train_params}

project_name = 'seq2seq_gru_attention'

encoder = encoders.BidirectionalGRUEncoder(**encoder_params).to(device)
decoder = decoders.GRUDecoderWithAttention(**decoder_params).to(device)

model = models.Seq2SeqGRUWithAttention(encoder, decoder, device)

print(model)
print('Number of parameters: {}\n'.format(sum([parameter.numel() for parameter in model.parameters()])))

criterion = nn.CrossEntropyLoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=train_params['lr'])

train_loader, test_loader = make_loaders(X, train_params['batch_size'])

#main_wandb(model, criterion, optimizer, device, train_loader, test_loader, params, project_name)
main_simple(model, criterion, optimizer, device, train_loader, test_loader, params)

In [ ]:
def display_attention(sentence, translation, attention):
    
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    
    attention = attention.squeeze(1).cpu().detach().numpy()
    a
    cax = ax.matshow(attention, cmap='bone')
   
    ax.tick_params(labelsize=15)
    
    #x_ticks = [''] + ['<sos>'] + [t.lower() for t in sentence] + ['<eos>']
    #y_ticks = [''] + translation
     
    #ax.set_xticklabels(x_ticks, rotation=45)
    #ax.set_yticklabels(y_ticks)

    #ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    #ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()
    plt.close()

reconstruction, attention = reconstruct_bassline(model, bassline)

In [ ]:
test_loss, test_acc = test(model, test_loader, criterion, device)
samples = model.sample()

for epoch in tqdm(range(train_params['N_epochs'])):

    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    validation_loss, validation_acc = test(model, validation_loader, criterion, device)
    
    model.decoder.update_teacher_forcing_ratio(decoder_params['teacher_forcing_ratio']/train_params['N_epochs'])
    
    if epoch+1 % 5:
        print('Epoch: {}, train_loss: {:.6f}, acc: {:.3f}'.format(epoch+1, train_loss, train_acc))

test_loss, test_acc = test(model, test_loader, criterion, device)
print('Test Loss After Training: {:.6f}'.format(test_loss))

In [7]:
x = torch.load('model_checkpoints/03_06__22_55.pt')

In [11]:
state_dict = utils.load_state_dict('03_06__22_55')

In [ ]:
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.eval()

# Simple Models

In [ ]:
K

In [ ]:
train_params = {'batch_size': 64,
               'N_epochs': 150,
               'lr': 5e-4,
               'teacher_forcing_ratio': 0.75}

encoder_params = {'input_size': K,
                  'embedding_size': 12,
                  'hidden_size': 16,
                  'n_layers': 1}

decoder_params = {'output_size': K,
                  'embedding_size': encoder_params['embedding_size'],
                  'hidden_size': encoder_params['hidden_size'],
                 'n_layers': encoder_params['n_layers']}

params = {'data': data_params,
          'encoder': encoder_params,
          'decoder':decoder_params, 
          'training':train_params}

project_name = 'seq2seq_gru_simple'

In [ ]:
with open('Seq2SeqGRUSimple_params.json', 'w') as outfile:
    json.dump(params, outfile, indent=4)

In [ ]:
encoder = encoders.GRUEncoder(**encoder_params)
decoder = decoders.SimpleGRUDecoder(**decoder_params)
model = models.Seq2SeqGRUSimple(encoder, decoder, device).to(device)

print(model)
print('Number of parameters: {}\n'.format(sum([parameter.numel() for parameter in model.parameters()])))

criterion = nn.CrossEntropyLoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=train_params['lr'])

train_loader, test_loader = make_loaders(X, train_params['batch_size'])

In [ ]:
#main_simple(model, criterion, optimizer, device, train_loader, test_loader, params)
main_wandb(model, criterion, optimizer, device, train_loader, test_loader, params, project_name)

In [ ]:
#encoder = encoders.LSTMEncoder(**encoder_params)
#decoder = decoders.SimpleLSTMDecoder(**decoder_params)
#model = models.Seq2SeqLSTM(encoder, decoder, device).to(device)

project_name = 'seq2seq_lstm_simple'

encoder = encoders.GRUEncoder(**encoder_params)
decoder = decoders.GRUDecoder(**decoder_params)
model = models.Seq2SeqGRU(encoder, decoder, device).to(device)

project_name = 'seq2seq_gru'